In [22]:
from PIL import Image
import numpy as np
from scipy.signal import convolve2d

def psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * np.log10(255.0 / np.sqrt(mse))

def gaussian_kernel(window_size, sigma=1.5):
    """Create a 2D Gaussian kernel."""
    ax = np.arange(-window_size // 2 + 1., window_size // 2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx ** 2 + yy ** 2) / (2. * sigma ** 2))
    return kernel / np.sum(kernel)

def ssim(img1, img2, window_size=11, K1=0.01, K2=0.03, L=255):
    """
    Compute the Structural Similarity Index (SSIM) between two images using NumPy.

    Args:
        img1: First image as a NumPy array (grayscale).
        img2: Second image as a NumPy array (grayscale).
        window_size: Size of the sliding window (default is 11).
        K1: Constant to stabilize the weak denominator (default is 0.01).
        K2: Constant to stabilize the weak denominator (default is 0.03).
        L: Dynamic range of the pixel values (default is 255 for 8-bit images).

    Returns:
        The SSIM index between the two images.
    """
    # Convert images to float64
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)

    # Constants to avoid division by zero
    C1 = (K1 * L) ** 2
    C2 = (K2 * L) ** 2

    # Window for local statistics (uniform)
    window = np.ones((window_size, window_size)) / (window_size ** 2)

    # Image dimensions
    M, N = img1.shape

    # Pad the images to deal with borders
    pad_size = window_size // 2
    img1_padded = np.pad(img1, pad_width=pad_size, mode='reflect')
    img2_padded = np.pad(img2, pad_width=pad_size, mode='reflect')

    # Initialize matrices for local statistics
    mu1 = np.zeros((M, N))
    mu2 = np.zeros((M, N))
    sigma1_sq = np.zeros((M, N))
    sigma2_sq = np.zeros((M, N))
    sigma12 = np.zeros((M, N))

    # Loop over each pixel to compute local statistics
    for i in range(M):
        for j in range(N):
            # Define the local region
            region1 = img1_padded[i:i + window_size, j:j + window_size]
            region2 = img2_padded[i:i + window_size, j:j + window_size]

            # Compute local means
            mu1[i, j] = np.sum(window * region1)
            mu2[i, j] = np.sum(window * region2)

            # Compute local variances and covariance
            sigma1_sq[i, j] = np.sum(window * (region1 - mu1[i, j]) ** 2)
            sigma2_sq[i, j] = np.sum(window * (region2 - mu2[i, j]) ** 2)
            sigma12[i, j] = np.sum(window * (region1 - mu1[i, j]) * (region2 - mu2[i, j]))

    # Compute SSIM map
    numerator1 = (2 * mu1 * mu2 + C1)
    numerator2 = (2 * sigma12 + C2)
    denominator1 = (mu1 ** 2 + mu2 ** 2 + C1)
    denominator2 = (sigma1_sq + sigma2_sq + C2)
    ssim_map = (numerator1 * numerator2) / (denominator1 * denominator2)

    # Average the SSIM map to get the final SSIM index
    ssim_index = np.mean(ssim_map)

    return ssim_index


def ssim(img1, img2, window_size=11, K1=0.01, K2=0.03, L=255):
    """
    Optimized SSIM implementation using vectorized operations and convolution.

    Args:
        img1: First image as a NumPy array (grayscale).
        img2: Second image as a NumPy array (grayscale).
        window_size: Size of the sliding window (default is 11).
        K1: Constant to stabilize the weak denominator (default is 0.01).
        K2: Constant to stabilize the weak denominator (default is 0.03).
        L: Dynamic range of the pixel values (default is 255 for 8-bit images).

    Returns:
        The SSIM index between the two images.
    """
    # Convert images to float64
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)

    # Constants to avoid division by zero
    C1 = (K1 * L) ** 2
    C2 = (K2 * L) ** 2

    # Window for local statistics (uniform)
    window = np.ones((window_size, window_size)) / (window_size ** 2)
    window = gaussian_kernel(window_size)

    # Compute mu1 and mu2 using convolution
    mu1 = convolve2d(img1, window, mode='same', boundary='symm')
    mu2 = convolve2d(img2, window, mode='same', boundary='symm')

    # Compute squares and products of mu1 and mu2
    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2
    mu1_mu2 = mu1 * mu2

    # Compute sigma1_sq, sigma2_sq, and sigma12 using convolution
    sigma1_sq = convolve2d(img1 * img1, window, mode='same', boundary='symm') - mu1_sq
    sigma2_sq = convolve2d(img2 * img2, window, mode='same', boundary='symm') - mu2_sq
    sigma12 = convolve2d(img1 * img2, window, mode='same', boundary='symm') - mu1_mu2

    # Compute SSIM map
    numerator1 = 2 * mu1_mu2 + C1
    numerator2 = 2 * sigma12 + C2
    denominator1 = mu1_sq + mu2_sq + C1
    denominator2 = sigma1_sq + sigma2_sq + C2
    ssim_map = (numerator1 * numerator2) / (denominator1 * denominator2)

    # Average the SSIM map to get the final SSIM index
    ssim_index = np.mean(ssim_map)
    return ssim_index

In [31]:
i1 = '../../qos-04/qos_04/fig/android_gray.png'
i2 = '../../qos-04/qos_04/fig/noisy_image_var0_1_mask0.001.png'
i2 = 'parrot_gray.png'

# Load images using PIL and convert to grayscale
img1 = Image.open(i1).convert('L')
img2 = Image.open(i2).convert('L')

# Convert images to NumPy arrays
img1 = np.array(img1)
img2 = np.array(img2)

print("PSNR: ", psnr(img1, img2))
print("SSIM: ", ssim(img1, img2))

PSNR:  7.449696530590884
SSIM:  0.24979164513684862
